In [37]:
import numpy as np, cv2

image = cv2.imread('images/test_img.jpg', cv2.IMREAD_COLOR)
logo = cv2.imread('images/logo.jpg', cv2.IMREAD_COLOR)
if image is None or logo is None: raise Exception('영상파일 읽기 오류')

masks = cv2.threshold(logo, 220, 255, cv2.THRESH_BINARY)[1]
masks = cv2.split(masks)

fg_pass_mask = cv2.bitwise_or(masks[0], masks[1])
fg_pass_mask = cv2.bitwise_or(masks[2], fg_pass_mask)
bg_pass_mask = cv2.bitwise_not(fg_pass_mask)

def onMouse(event, x, y, flags, param):
    global image, logo, fg_pass_mask, bg_pass_mask

    if event == cv2.EVENT_LBUTTONDOWN:
        img = image.copy()

        (H, W), (h, w) = image.shape[:2], logo.shape[:2]
        
        if x < 0 or y < 0 or x+w > W or y+h > H:
            return

        roi = img[y:y+h, x:x+w]

        foreground = cv2.bitwise_and(logo, logo, mask=fg_pass_mask)
        background = cv2.bitwise_and(roi, roi, mask=bg_pass_mask)
        
        dst = cv2.add(background, foreground)
        img[y:y+h, x:x+w] = dst

        cv2.imshow('image', img)

cv2.imshow('image', image)
cv2.setMouseCallback('image', onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()